In [1]:
import numpy as np


In [ ]:
import panda as pd


In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
movies = pd.read_csv(r'C:\Users\sonuk\Desktop\tmdb_5000_movies.csv')
credits = pd.read_csv(r'C:\Users\sonuk\Desktop\tmdb_5000_credits.csv') 

In [ ]:
movies.head()

In [ ]:
movies.shape

In [ ]:
credits.head()

In [ ]:
movies.head()
# budget
# homepage
# id
# original_language
# original_title
# popularity
# production_comapny
# production_countries
# release-date(not sure)

In [ ]:
movies.head()


In [ ]:
print("Movies columns:", movies.columns)
print("Credits columns:", credits.columns)


In [ ]:

movies = movies.merge(
    credits,
    left_on='id',
    right_on='movie_id',
    suffixes=('_movie', '_credit')
)


In [ ]:
print(movies.columns)


In [ ]:
movies_clean = movies[['id', 'title', 'overview', 'genres', 'keywords', 'cast_credit', 'crew_credit']]


In [ ]:
# sirf required columns select karo
movies = movies[['id', 'title', 'overview', 'genres', 'keywords', 'cast_credit', 'crew_credit']]

# columns rename kar do (optional)
movies.rename(columns={'cast_credit':'cast', 'crew_credit':'crew'}, inplace=True)


In [ ]:
movies.head()

In [ ]:
import ast

In [ ]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies['genres'] = movies['genres'].apply(convert)
movies.head()

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)
movies.head()

In [ ]:
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [ ]:
movies['cast'] = movies['cast'].apply(convert)
movies.head()

In [ ]:
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

In [ ]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)


In [ ]:
movies.sample(5)

In [ ]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [ ]:
movies.head()

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
#new.head()

In [ ]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
    

In [ ]:
vector = cv.fit_transform(new['tags']).toarray()

In [ ]:
vector.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity

In [ ]:
new[new['title'] == 'The Lego Movie'].index[0]


In [ ]:
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)
        
    

In [ ]:
recommend('Gandhi')

In [ ]:
import pickle

In [ ]:
pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
import os
print(os.getcwd())


In [ ]:
import pickle
import os

# Check path
print(os.getcwd())  # Tumhare case me ye C:\Users\sonuk\PycharmProjects\movie-recommender-system hona chahiye

# Load files
movies_path = 'model/movie_list.pkl'
similarity_path = 'model/similarity.pkl'

if os.path.exists(movies_path) and os.path.exists(similarity_path):
    movies = pickle.load(open(movies_path,'rb'))
    similarity = pickle.load(open(similarity_path,'rb'))
    print("Pickle files loaded successfully!")
    print(movies.columns)  # columns check karo
else:
    print("Pickle files missing!")


In [ ]:
print(movies.columns)


In [ ]:
print(movies.columns)



In [ ]:
movies = movies[['id', 'title_x', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies = movies.rename(columns={'title_x':'title'})


In [ ]:
movies.to_pickle("model/movie_list.pkl")


In [ ]:
import os

# Agar model folder exist nahi karta, to create karo
if not os.path.exists('model'):
    os.makedirs('model')


In [ ]:
movies.to_pickle("model/movie_list.pkl")


In [ ]:
import os
import pandas as pd
import pickle

# Ensure model folder exists
if not os.path.exists('model'):
    os.makedirs('model')

# CSV load
movies = pd.read_csv(r"C:\Users\sonuk\Desktop\tmdb_5000_movies.csv")
credits = pd.read_csv(r"C:\Users\sonuk\Desktop\tmdb_5000_credits.csv")

# Merge
movies = movies.merge(credits, left_on='id', right_on='movie_id')

# Select required columns & rename
movies = movies[['id', 'title_x', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies = movies.rename(columns={'title_x':'title'})

# Save movie_list.pkl
movies.to_pickle("model/movie_list.pkl")

# Create dummy similarity matrix for now
import numpy as np
similarity = np.identity(len(movies))  # real similarity calculate karna baad me
pickle.dump(similarity, open("model/similarity.pkl", "wb"))

print("Pickle files saved successfully!")


In [ ]:
import streamlit as st
import pickle
import requests

# Load pickle
movies = pickle.load(open('model/movie_list.pkl','rb'))
similarity = pickle.load(open('model/similarity.pkl','rb'))

# Function to fetch poster from TMDB
def fetch_poster(movie_id):
    api_key = "8265bd1679663a7ea12ac168da84d2e8"  # apni TMDB API key
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&language=en-US"
    data = requests.get(url).json()
    poster_path = data.get('poster_path')
    if poster_path:
        return "https://image.tmdb.org/t/p/w500" + poster_path
    else:
        return "https://via.placeholder.com/500x750?text=No+Image"

# Recommendation function
def recommend(movie):
    index = movies[movies['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    recommended_movie_names = []
    recommended_movie_posters = []
    for i in distances[1:6]:
        movie_id = movies.iloc[i[0]].id
        recommended_movie_names.append(movies.iloc[i[0]].title)
        recommended_movie_posters.append(fetch_poster(movie_id))
    return recommended_movie_names, recommended_movie_posters

# Streamlit UI
st.header("Movie Recommender System")

movie_list = movies['title'].values
selected_movie = st.selectbox("Type or select a movie", movie_list)

if st.button('Show Recommendation'):
    names, posters = recommend(selected_movie)
    col1, col2, col3, col4, col5 = st.columns(5)
    for i, col in enumerate([col1, col2, col3, col4, col5]):
        col.text(names[i])
        col.image(posters[i])


In [ ]:
pip install streamlit


In [ ]:
!streamlit --version


In [ ]:
st.write(selected_movie)
index = movies[movies[title_col] == selected_movie].index[0]
st.write("Index:", index)
st.write("Distances:", sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1]))


In [ ]:
import pickle

# Just check if files exist and can be loaded
try:
    with open('model/movie_list.pkl', 'rb') as f:
        movies = pickle.load(f)
        print("Movies file loaded!")
        print(f"Type: {type(movies)}")
        if hasattr(movies, 'columns'):
            print(f"Columns: {movies.columns.tolist()}")
            
    with open('model/similarity.pkl', 'rb') as f:
        similarity = pickle.load(f)
        print("Similarity file loaded!")
        print(f"Type: {type(similarity)}")
        print(f"Shape: {len(similarity)} if list, or proper shape if numpy array")
        
except Exception as e:
    print(f"Error: {e}")


In [ ]:
import pandas as pd
import numpy as np
import pickle
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

# CSV load
movies = pd.read_csv(r"C:\\Users\\sonuk\\Desktop\\tmdb_5000_movies.csv")
credits = pd.read_csv(r"C:\\Users\\sonuk\\Desktop\\tmdb_5000_credits.csv")

# Merge datasets
movies = movies.merge(credits, left_on='id', right_on='movie_id')

# Select required columns
movies = movies[['id', 'title_x', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies = movies.rename(columns={'title_x': 'title'})

# Drop NaN values
movies.dropna(inplace=True)

# Preprocessing functions
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L

def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter += 1
    return L

def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ", ""))
    return L1

# Apply preprocessing
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert3)  # Only top 3 cast members
movies['crew'] = movies['crew'].apply(fetch_director)  # Only directors

# Remove spaces
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

# Create tags
movies['overview'] = movies['overview'].apply(lambda x: x.split())
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))

# Create new dataframe with only necessary columns
new_df = movies[['id', 'title', 'tags']]

# Vectorization
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()

# Calculate REAL similarity matrix
similarity = cosine_similarity(vectors)

# Ensure model folder exists
if not os.path.exists('model'):
    os.makedirs('model')

# Save files
new_df.to_pickle("model/movie_list.pkl")
pickle.dump(similarity, open("model/similarity.pkl", "wb"))

print("Files created successfully!")
print(f"Movies shape: {new_df.shape}")
print(f"Similarity shape: {similarity.shape}")

In [ ]:
import pickle
import os

BASE_DIR = r"C:\Users\sonuk\PycharmProjects\movie-recommender-system"
movies = pickle.load(open(os.path.join(BASE_DIR, 'model/movie_list.pkl'), 'rb'))
similarity = pickle.load(open(os.path.join(BASE_DIR, 'model/similarity.pkl'), 'rb'))

# Check head of movies
print(movies.head())

# Check similarity matrix shape
print(len(similarity), len(similarity[0]))


In [ ]:
import requests

TMDB_API_KEY = "8265bd1679663a7ea12ac168da84d2e8"

# Test first movie
movie_name = movies['title'][0]  # Avatar
index = movies[movies['title'] == movie_name].index[0]
movie_id = movies.iloc[index]['id']

print("Movie:", movie_name)
print("Movie ID:", movie_id)

# Fetch poster
url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={TMDB_API_KEY}&language=en-US"
response = requests.get(url)
print("Status code:", response.status_code)
print(response.json())  

poster_path = response.json().get('poster_path')
if poster_path:
    poster_url = "https://image.tmdb.org/t/p/w500/" + poster_path
else:
    poster_url = "https://via.placeholder.com/500x750?text=No+Image"

print("Poster URL:", poster_url)


In [ ]:
import requests

TMDB_API_KEY = "8265bd1679663a7ea12ac168da84d2e8"

movie_id = 19995  # Avatar
url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={TMDB_API_KEY}&language=en-US"

response = requests.get(url)
print("Status code:", response.status_code)
data = response.json()
print(data)  # TMDB API response check

poster_path = data.get('poster_path')
if poster_path:
    poster_url = "https://image.tmdb.org/t/p/w500/" + poster_path
else:
    poster_url = "https://via.placeholder.com/500x750?text=No+Image"

print("Poster URL:", poster_url)
